# Pipeline RAG - OuvidorIA

Este notebook implementa o núcleo de inteligência do **OuvidorIA**. Aqui configuramos o fluxo de **RAG (Retrieval-Augmented Generation)** que permite ao assistente responder perguntas com base nos manuais oficiais.

**Etapas:**
1.  **Ingestão:** Carregamento de textos (simulando manuais do Fala.BR).
2.  **Indexação:** Criação de Embeddings e armazenamento no Qdrant.
3.  **Retrieval (Recuperação):** Busca semântica dos trechos relevantes.
4.  **Generation (Geração):** Criação de respostas com LLM (Gemini).
5.  **Testes:** Validação do fluxo com perguntas de usuários.

---

In [ ]:
!pip install llama-index llama-index-llms-google-genai llama-index-embeddings-huggingface llama-index-vector-stores-qdrant qdrant-client -q
!pip install -U "llama-index" "llama-index-llms-google-genai" "google-genai"

In [ ]:
import os
import qdrant_client
from llama_index.core import VectorStoreIndex, Document, Settings, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.llms.google_genai import GoogleGenerativeAI # Import atualizado
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.google_genai import GoogleGenAI

# --- CONFIGURAÇÃO ---
GOOGLE_API_KEY = ""
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

import importlib, pkgutil
modname = "llama_index.llms.google_genai"
m = importlib.import_module(modname)
print("m.__file__:", getattr(m, "__file__", "unknown"))
print("exports:", [n for n in dir(m) if not n.startswith("_")])

# 1. Configuração do Modelo de Embeddings (Gratuito/Open Source para economizar custos)
# Usaremos um modelo multilíngue leve da HuggingFace
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 2. Configuração da LLM (Gemini 1.5 Flash - Rápido e Eficiente)
# Usando a classe GoogleGenerativeAI compatível com os novos modelos
Settings.llm = GoogleGenerativeAI(model="models/gemini-1.5-flash", temperature=0.2)

embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

llm = GoogleGenAI(
    model="models/gemini-1.5-flash",   # ou "gemini-1.5-flash" conforme o fornecedor
    api_key=os.environ.get("GOOGLE_API_KEY"),
    temperature=0.2
)

# Exemplo mínimo de uso
resp = llm.complete("Escreva um haicai sobre programação")
print(resp)

ModuleNotFoundError: No module named 'qdrant_client'

## 1. Criação da Base de Conhecimento (Simulação)
Como não estamos carregando PDFs externos neste ambiente de teste, criaremos documentos manuais com informações reais extraídas do **Manual do Fala.BR**.

In [ ]:
docs_text = [
    # Documento 1: Definições Básicas
    Document(
        text="""
        O que é uma Ouvidoria?
        A ouvidoria é um canal de interlocução entre o cidadão e a Administração Pública.
        Sua finalidade é receber manifestações dos cidadãos, analisar, orientar e encaminhar às áreas responsáveis pelo tratamento ou apuração, respondendo ao manifestante.
        A ouvidoria atua no pós-atendimento, ou seja, quando o cidadão já procurou o serviço e não obteve resposta ou a resposta não foi satisfatória.
        """,
        metadata={"fonte": "Manual de Ouvidoria Pública", "capitulo": "Introdução"}
    ),
    # Documento 2: Tipos de Manifestação
    Document(
        text="""
        Tipos de Manifestação no Fala.BR:
        1. Denúncia: Comunicação de prática de ato ilícito cuja solução dependa da atuação de órgão de controle. Ex: uso indevido de carro oficial.
        2. Reclamação: Demonstração de insatisfação relativa a serviço público. Ex: demora no atendimento, falta de remédio.
        3. Solicitação: Requerimento de adoção de providência por parte da Administração. Ex: conserto de buraco na rua.
        4. Sugestão: Proposta de aprimoramento de políticas e serviços públicos.
        5. Elogio: Demonstração de satisfação ou reconhecimento.
        """,
        metadata={"fonte": "Manual Fala.BR", "capitulo": "Tipos de Manifestação"}
    ),
    # Documento 3: Prazos
    Document(
        text="""
        Prazos para Resposta:
        O prazo para resposta às manifestações é de 30 (trinta) dias, prorrogável por mais 30 (trinta) dias, mediante justificativa expressa.
        Para pedidos de Acesso à Informação (LAI), o prazo é de 20 (vinte) dias, prorrogável por mais 10 (dez) dias.
        """,
        metadata={"fonte": "Lei 13.460/2017", "topico": "Prazos"}
    )
]

print(f"Documentos preparados: {len(docs_text)}")

## 2. Indexação Vetorial (Qdrant)
Aqui inicializamos o cliente Qdrant (em memória para este teste) e criamos o índice vetorial.

In [ ]:
# Inicializa cliente Qdrant em memória (RAM)
client = qdrant_client.QdrantClient(location=":memory:")

# Cria o Vector Store
vector_store = QdrantVectorStore(client=client, collection_name="ouvidoria_knowledge")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Indexa os documentos (Chunking -> Embedding -> Upsert no Qdrant)
index = VectorStoreIndex.from_documents(
    docs_text,
    storage_context=storage_context,
)

print("Indexação concluída com sucesso no Qdrant!")

## 3. Implementação da Lógica de Retrieval & Generation
Configuramos o *Query Engine* com um prompt personalizado para garantir que o chatbot aja como um especialista em ouvidoria.

In [ ]:
# Definindo o Template do Prompt (System Prompt)
qa_prompt_str = (
    "Você é o OuvidorIA, um assistente especializado em ajudar cidadãos com a Ouvidoria Pública.\n"
    "Use as informações de contexto abaixo para responder à dúvida do cidadão de forma clara, educada e direta.\n"
    "Se a resposta não estiver no contexto, diga que não sabe e oriente a buscar o site oficial da CGU.\n"
    "---------------------\n"
    "CONTEXTO:\n"
    "{context_str}\n"
    "---------------------\n"
    "PERGUNTA DO CIDADÃO: {query_str}\n"
    "RESPOSTA DO OUVIDORIA:"
)

# Configurando o mecanismo de busca
query_engine = index.as_query_engine(
    similarity_top_k=2, # Busca os 2 trechos mais relevantes
    text_qa_template=PromptTemplate(qa_prompt_str)
)

## 4. Testes do Fluxo (Scripts de Validação)
Vamos simular perguntas reais de usuários para validar se o RAG está recuperando o contexto correto e gerando boas respostas.

In [ ]:
def testar_rag(pergunta):
    print(f"\nPERGUNTA: {pergunta}")
    print("-" * 50)

    # Executa a pipeline
    response = query_engine.query(pergunta)

    # Exibe a resposta gerada
    print(f"RESPOSTA: {response}\n")

    # Debug: Mostra quais documentos foram usados (Retrieval)
    print("FONTES RECUPERADAS:")
    for node in response.source_nodes:
        score = "{:.2f}".format(node.score)
        fonte = node.metadata.get('fonte', 'N/A')
        trecho = node.text[:100] + "..." # Primeiros 100 caracteres
        print(f"   - [Score: {score}] Fonte: {fonte} | Trecho: \"{trecho}\"")

# --- Cenários de Teste ---

# Cenário 1: Dúvida sobre conceito
testar_rag("Qual a diferença entre denúncia e reclamação?")

# Cenário 2: Dúvida sobre prazos
testar_rag("Quanto tempo a ouvidoria tem para me responder?")

# Cenário 3: Pergunta fora do contexto (Teste de Alucinação)
testar_rag("Qual a receita de bolo de cenoura?")